In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from time import sleep
import re
import random

In [2]:
url="http://wenshu.court.gov.cn/website/wenshu/181107ANFZ0BXSK4/index.html?docId=7418e8ce379b41a6bf85ab34009c7c15"
#s=requests.Session()
headers = {
    "Cookies":"Hm_lvt_9e03c161142422698f5b0d82bf699727=1578373175,1578373207,1578373259,1578373713; HM4hUBT0dDOn80S=JBnYSuVdpUpBw7c0mFeeO9aUsgAWyWaGz_Jm.gTyTqxSq705ljAKJNQ1ToSEnLHM; SESSION=d75bbf0d-d81d-45cc-bc39-85f73ae75625; wzws_cid=e75f839ab2a7707f278f5b72e1fe3d4532b471c4993d35a98f6414b66299a1a7ed361d014111db71343a798bc7e5957c39ccc183330bb290ebf93662382d90dda4359906d37d489f625a5edfd68fb242; HM4hUBT0dDOn80T=4twUJIEbWPce1rG9EqNSAiRjRbO87Hzk9yRLdosYOZ1dG8fiDF9dU8RId93gOT_2Q4_Ibw9ujUwU_Eou9IViZgGwrrne1V7Yn7Zlpy2CvaPvAhD8o9zWKqpduVxZYyt7R82JKwedDDu8J2GX5wb.c5a8i2MZpbSg48iHoj06Hs4mCPqhyPjxGRs2gXG0K9zkucJjBVpveO8_CV5dS0t8082Uia7DeZICYTubPpI9QxAHPGfUo0zyivRnPwyMkQqyvB4a.1FHHRHtyKjKsi85zj8pRZ40siGR0.4bzeGFjHUGCUmu0pfpat6DqZMyW_bspN6JuxytbT45Qo95nFgEbccvCb9jdZ8EC7fUOa.yIXrTiR6Deah9DP1ApBInY_Cg7.zL",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36"
}


In [9]:
def get_page():#获取页面信息
    html=driver.page_source
    soup=BeautifulSoup(html,features="lxml")
    text=[i.get_text("|",strip=True) for i in soup.find_all(class_="PDF_pox")]
    if len(text)==0:
        return 
    text=text[0].replace("|对赌协议|","对赌协议").split("|")
    text.append(soup.find(class_="PDF_title").get_text())
    return text

In [4]:
def switch_get_info():#切换至其他页面并调用get_page获取页面信息
    info=[]
    curren_page=driver.current_window_handle
    handles=driver.window_handles
    handles.remove(curren_page)
    for i in handles:
        sleep(random.uniform(0,1))
        driver.switch_to.window(i)
        text=get_page()
        info.append(text)
        driver.close()
    return info
#switch_get_info()

In [5]:
def click_event(page_num):#模拟点击事件
    sleep(random.uniform(1,3))
    for  i in driver.find_elements_by_class_name("caseName"): 
        try:
            i.click()
        except:
            pass
    info = switch_get_info()
    driver.switch_to.window(driver.window_handles[0])
    try:
        #driver.find_element_by_xpath("//*[text()='下一页']").click()
        driver.find_element_by_xpath(".//*[@value=%i]"%page_num).click()
    except:
        print(i)
        pass
    return info

In [6]:
driver=webdriver.Chrome(r"C:\Users\HASEE\chromedriver.exe")
driver.get("https://www.google.com/")

In [30]:
html=get_page()

In [10]:
driver.switch_to.window(driver.window_handles[0])

In [11]:
information=[]

In [13]:
for i in range(2,56):
    information.append(click_event(i))

In [12]:
import copy
copy_info=information.copy()

In [14]:
check=[]
new_info=[]
for i in range(len(information)):
    for j in range(len(information[i])):
        try:
            if information[i][j][2] not in check:
                check.append(information[i][j][2])
                new_info.append(information[i][j])
            else:
                continue
        except:
            pass

In [15]:
len(new_info)

463

In [160]:
data_list=[]

for i in range(len(new_info)):
    data={}
    data["案号"]=new_info[i][2]
    #info =new_info[i][j.split("")]
    for j in range(len(new_info[i])):
        if "原告" in new_info[i][j] and "某"in new_info[i][j]:
            if data.get("原告",False)==False:
                data["原告"]=new_info[i][j]
        elif "被告" in new_info[i][j] and "某" in new_info[i][j]:
            if data.get("被告",False)==False:                
                data["被告"]=new_info[i][j]
    for j in range(len(new_info[i])):
        if "诉讼请求"in new_info[i][j] and "事实和理由"in new_info[i][j]:
            info=new_info[i][j].split("。")
            if data.get("诉讼请求",False)==False:
                data["诉讼请求"]=info[0]
                data["事实和理由"]=''.join(info[1:])
        elif "诉讼请求" in new_info[i][j]:
            if data.get("诉讼请求",False)==False:
                data["诉讼请求"]=new_info[i][j]
        elif "辩称" in new_info[i][j] and "被告"in new_info[i][j]:
            if data.get("被告辩称",False)==False:
                data["被告辩称"]=new_info[i][j]
        elif "事实" in new_info[i][j] and"理由" in new_info[i][j]:
            data["事实和理由"]=new_info[i][j]
        elif "经审理"  in  new_info[i][j] or "经审查" in new_info[i][j] or "本院认为" in new_info[i][j]:
            if data.get("法院审理意见",False)==False:
                data["法院审理意见"]="".join(new_info[i][j:])
    data["全文"]="".join(new_info[i])
    data_list.append(data)


In [16]:
def get_data_list(new_info):
    data_list=[]
    for i in range(len(new_info)):
        data={}
        if "一审" in new_info[i][-1]:
            data["判决书全称"]=new_info[i][-1]
            data["法院"]=new_info[i][0]
            data["案号"]=new_info[i][2]
            for j  in range(len(new_info[i])):
                if "原告" in new_info[i][j]:
                    if data.get("原告",False)==False:
                        data["原告"]=new_info[i][j]
                elif "被告" in new_info[i][j]:
                    if data.get("被告",False)==False:
                        data["被告"]=new_info[i][j]
                elif "本院认为" in new_info[i][j]:
                    if data.get("本院认为",False)==False:
                        data["本院认为"]=new_info[i][j]
                elif "综上" in new_info[i][j] :
                    if data.get("综上",False)==False:
                        data["结果"]=new_info[i][j]
            for j in new_info[i][len(new_info[i])-5:]:
                if "年" in j :
                    data["结案日期"]=j
            data["全文"]="".join(new_info[i])
        else:
            data["判决书全称"]=new_info[i][-1]
            data["法院"]=new_info[i][0]
            data["案号"]=new_info[i][2]
            for j in range(len(new_info[i])):
                if "原审原告" in new_info[i][j] or "一审原告" in new_info[i][j]:
                    if data.get("原审原告",False)==False:
                        data["原审原告"]=new_info[i][j]
                    else:
                        data["原审原告"]+=new_info[i][j]
                elif "原审被告" in new_info[i][j] or "一审被告" in new_info[i][j]:
                    if data.get("原审被告",False)==False:
                        data["原审被告"]=new_info[i][j]
                    else:
                        data["原审被告"]+=new_info[i][j]
                elif"本院认为" in new_info[i][j]:
                    if data.get("本院认为",False)==False:
                        data["本院认为"]=new_info[i][j]
                elif "综上" in new_info[i][j] :
                    if data.get("综上",False)==False:
                        data["结果"]=new_info[i][j]
            for j in new_info[i][len(new_info[i])-5:]:
                if "年" in j :
                    data["结案日期"]=j
            data["全文"]="".join(new_info[i])
        data_list.append(data)
    return data_list

In [52]:
for j in new_info[3][len(new_info[3])-5:]:
    if "年" in j:
        print(j)

二〇一八年九月七日


In [17]:
data_list=get_data_list(new_info)

In [18]:
data_list[1]

{'判决书全称': '  陆芸芸等与北京四方继保自动化股份有限公司二审民事判决书',
 '法院': '北京市高级人民法院',
 '案号': '（2019）京民终252号',
 '原审被告': '上诉人（原审被告）：方强，男，1966年3月14日出生，汉族，住上海市闵行区。上诉人（原审被告）：陆芸芸，女，1966年7月18日出生，汉族，住上海市闵行区。',
 '原审原告': '被上诉人（原审原告）：北京四方继保自动化股份有限公司，住所地北京市海淀区上地信息产业基地四街9号。',
 '结果': '综上所述，方强、陆芸芸的上诉请求及理由，依法均不成立，依法应予驳回。一审判决认定事实清楚，适用法律正确，应予维持。本院依照《中华人民共和国民事诉讼法》第一百七十条第一款第（一）项之规定，判决如下：',
 '本院认为': '本院二审审理期间，上诉人方强、上诉人陆芸芸及被上诉人四方继保公司均提交了新证据。方强、陆芸芸共同提交的证据1．泓申公司分别于2016年6月27日、2018年1月17日、2018年4月9日向中证天通会计师事务所支付审计费共计6．5万元凭证；泓申公司与中证天通会计师事务所签订的《审计业务约定书》。证据2．四方智能（武汉）控制技术有限公司企业信用信息公示报告。证据3．四方继保公司2017年年度报告（节选）。四方继保公司提交的证据1．中证天通会计师事务所出具的中证天通（2017）审字第0201258号《上海泓申科技发展有限公司2016年度财务报表之审计报告》（以下简称《泓申公司2016年审计报告》）；证据2．中证天通会计师事务所出具的中证天通（2018）审字第0201163号《上海泓申科技发展有限公司2017年度财务报表之审计报告》（以下简称《泓申公司2017年审计报告》）；证据3．泓申公司于2018年1月17日向中证天通会计师事务所支付的2万元审计费凭证及发票；证据4．泓申公司与中证天通会计师事务所签订的《审计业务约定书》及泓申公司于2018年4月9日向中证天通会计师事务所支付2．5万元审计费凭证及发票。本院组织双方当事人进行了证据交换和质证。四方继保公司对方强、陆芸芸提交的证据的真实性均予认可；方强、陆芸芸认可四方继保公司提交的证据3、证据4的真实性，认可证据1、证据2的复印件与原件一致，但以四方继保公司未在一审提交该两份审计报告，且中证天通会计师事务所还同时出具

In [19]:
import pandas as pd
df=pd.DataFrame(data=data_list)
#df.to_csv("法院信息.csv",encoding="utf-8")

In [168]:
df.to_csv("法院信息2012.csv",encoding="utf-8")

In [164]:
df.iloc[4]["全文"]

'上海市松江区人民法院民 事 判 决 书（2013）松少民初字第57号原告仇某，男，2005年10月20日生，汉族，住上海市松江区。法定代理人仇某某，男，1961年4月6日生，汉族，住上海市松江区。委托代理人王娜玉，上海市诚至信律师事务所律师。被告张金花，女，1971年11月20日生，汉族，住上海市松江区。原告仇某诉被告张某某抚养费纠纷一案，本院于2013年11月15日立案受理后，依法适用简易程序于2013年12月26日公开开庭进行了审理。原告仇某的法定代理人仇某某及其委托代理人王娜玉到庭参加诉讼。被告张金花经本院合法传唤，无正当理由拒不到庭，本院依法缺席审理。本案现已审理终结。原告仇某诉称：原告父亲与被告协议离婚后，原告随父亲共同生活，被告每月给付生活费500元，并支付其他费用。自2011年11月起，被告一直以种种理由拒绝支付抚养费。故原告起诉要求被告自2011年11月起每月给付原告生活费500元，教育费、医疗费由被告负担50%，至原告满18周岁止。被告张金花未作答辩。经审理查明：原告与被告系母子关系。原告之父仇某某与被告于2011年7月26日协议离婚，双方约定：原告随父亲仇某某共同生活，被告每月给付生活费500元（随着社会生活水平提高而增加），教育费由被告承担50%，至原告满18周岁止。之后，被告支付了生活费1，000元，教育费1，500元，保险费1，000元。自2011年11月起分文未付，遂涉讼。以上事实，有离婚证、自愿离婚协议书、出生证及当事人陈述等证据证实。本院认为，父母对子女有抚养教育的义务。离婚后，一方抚养子女的，另一方应负担必要的抚养费的部分或全部，负担费用的多少和期限的长短，由双方协议；协议不成时，由人民法院判决。子女抚养费的具体数额，可根据子女的实际需要、父母双方的负担能力和当地的实际生活水平予以确定。本案原告要求被告自2011年11月起支付原告生活费及教育费、医疗费的50%，并无不当，本院予以支持。综上，依照《中华人民共和国婚姻法》第二十一条第一款、第三十七条第一款、《中华人民共和国民事诉讼法》第一百四十四条之规定，判决如下：被告张某某自2011年11月起每月给付原告仇也生活费500元，医疗费、教育费凭有效票据承担50%，至原告仇也满18周岁止。如果付款义务人未按本判决指定的期间履行给付金钱义务，应当依照《中华人民共和国民事诉讼法》第二百五十三

In [20]:
df

,判决书全称,法院,案号,原审原告,原审被告,结果,本院认为,全文,结案日期,原告,被告
0,新余甄投云联成长投资管理中心、广东运货柜信息技术有限公司新增资本认购纠纷、买卖合同纠纷二...,江西省高级人民法院,（2019）赣民终178号,上诉人（原审原告）：新余甄投云联成长投资管理中心（有限合伙），住所地：江西省新余市仙女湖区仰...,被上诉人（原审被告）：广东运货柜信息技术有限公司，住所地：广东省广州市高新技术产业开发区科学...,综上所述，甄投中心的上诉请求不能成立，应予驳回。一审判决认定事实清楚，适用法律正确，应予维持...,本院认为，本案二审争议的问题是：1、股权回购条款是否有效，运货柜公司、龙科是否需要承担股权回...,江西省高级人民法院民 事 判 决 书（2019）赣民终178号上诉人（原审原告）：新余甄投云...,NaN,NaN,NaN
1,陆芸芸等与北京四方继保自动化股份有限公司二审民事判决书,北京市高级人民法院,（2019）京民终252号,被上诉人（原审原告）：北京四方继保自动化股份有限公司，住所地北京市海淀区上地信息产业基地四街9号。,上诉人（原审被告）：方强，男，1966年3月14日出生，汉族，住上海市闵行区。上诉人（原审被...,综上所述，方强、陆芸芸的上诉请求及理由，依法均不成立，依法应予驳回。一审判决认定事实清楚，适...,本院二审审理期间，上诉人方强、上诉人陆芸芸及被上诉人四方继保公司均提交了新证据。方强、陆芸芸...,北京市高级人民法院民 事 判 决 书（2019）京民终252号上诉人（原审被告）：方强，男，...,二〇一九年八月六日,NaN,NaN
2,金燕与建银文化产业股权投资基金（天津）有限公司合同纠纷二审民事判决书,北京市高级人民法院,（2018）京民终18号,被上诉人（原审原告）：建银文化产业股权投资基金（天津）有限公司。,上诉人（原审被告）：金燕，女，1969年1月14日出生，汉族，北京正在发生文化传媒有限公司总...,综上所述，金燕的上诉请求不能成立，应予驳回；一审判决认定事实基本清楚，适用法律正确，应予维持...,针对证据1，金燕解释亚洲文化企业有限公司是在美国注册成立的，证据中列明在美国的雇主识别号码；...,北京市高级人民法院民 事 判 决 书（2018）京民终18号上诉人（原审被告）：金燕，女，1...,二〇一九年十月二十二日,NaN,NaN
3,大同华盛昌泰投资管理有限公司、黄某与珠海晋鲁股权投资有限公司、丁某等合同纠纷二审民事判决书,山西省高级人民法院,（2019）晋民终65号,上诉人（原审原告）：大同华盛昌泰投资管理有限公司。上诉人（原审原告）：黄某，男，1970年1...,被上诉人（原审被告）：珠海晋鲁股权投资有限公司。被上诉人（原审被告）：丁某，男，1986年1...,综上，原审判决认定事实清楚，适用法律正确，应予维持。上诉人的上诉缺乏充足事实依据以及法律依据...,本院认为，针对上诉人的上诉以及被上诉人的答辩意见，本案的焦点问题为：（一）上诉人与被上诉人于...,山西省高级人民法院民事判决书（2019）晋民终65号上诉人（原审原告）：大同华盛昌泰投资管理...,二〇一九年七月十八日,NaN,NaN
4,河北华安生物医药股份有限公司、以岭医药科技有限公司新增资本认购纠纷、买卖合同纠纷再审审查...,中华人民共和国最高人民法院,（2018）最高法民申5504号,被申请人（一审原告、二审被上诉人）：以岭医药科技有限公司，住所地西藏自治区拉萨市堆龙德庆区世...,再审申请人（一审被告、二审上诉人）：河北华安生物医药股份有限公司，住所地河北省石家庄市高新区...,综上，华安公司的申请再审事由不符合《中华人民共和国民事诉讼法》第二百条第一项、第六项、第八项...,NaN,中华人民共和国最高人民法院民 事 裁 定 书（2018）最高法民申5504号再审申请人（一审...,二〇一八年十二月二十九日,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
458,薛惠璇与郭丹公司增资纠纷一审民事判决书,北京市海淀区人民法院,（2014）海民初字第10678号,NaN,NaN,综上，本院依照《中华人民共和国合同法》第八条、第一百零九条之规定，判决如下：,本院认为：薛惠璇与郭丹、盛天公司三方签订的投资备忘录中，体现了薛惠璇向盛天公司增资入股的共同...,北京市海淀区人民法院民 事 判 决 书（2014）海民初字第10678号原告薛惠璇，男。委托...,二〇一四年七月二十八日,原告薛惠璇，男。,被告郭丹，女。
459,贺昤与郭丹公司增资纠纷一审民事判决书,北京市海淀区人民法院,（2014）海民初字第10675号,NaN,NaN,综上，本院依照《中华人民共和国合同法》第八条、第一百零九条之规定，判决如下：,本院认为：贺昤与郭丹、盛天公司三方签订的投资备忘录中，体现了贺昤向盛天公司增资入股的共同意思...,北京市海淀区人民法院民 事 判 决 书（2014）海民初字第10675号原告贺昤，男。委托代...,二〇一四年七月二十八日,原告贺昤，男。,被告郭丹，女。
460,江中与郭丹公司增资纠纷一审民事判决书,北京市海淀区人民法院,（2014）海民初字第10677号,NaN,NaN,综上，本院依照《中华人民共和国合同法》第八条、第一百零九条之规定，判决如下：,本院认为：江中与郭丹、盛天公司三方签订的投资备忘录中，体现了江中向盛天公司增资入股的共同意思...,北京市海淀区人民法院民 事 判 决 书（2014）海民初字第10677号原告江中，男。委托代...,二〇一四年七月二十八日,原告江中，男。,被告郭丹，女。
461,宁波东元创业投资有限公司与张才军、张才富与公司有关的纠纷一审民事判决书,四川省绵阳市涪城区人民法院,（2015）涪民初字第6087号,NaN,NaN,综上所述，为维护当事人的合法权益，根据《中华人民共和国合同法》第六十条、第一百零七条、第一百...,NaN,四川省绵阳市涪城区人民法院民 事 判 决 书（2015）涪民初字第6087号原告：宁波东元创...,二〇一六年九月十三日,原告：宁波东元创业投资有限公司，住所地：宁波市科技园区创业大厦3-29-1室。,被告：张才军，男，生于1969年3月1日，汉族，四川绵阳重业齿轮有限责任公司董事长，住四川省...


In [21]:
df=df[["判决书全称",'法院','案号','原告','被告','原审原告','原审被告','本院认为','结果','结案日期',"全文"]]

In [22]:
df.to_csv("对赌协议.csv",encoding="utf-8")